In [2]:
import pandas as pd
import numpy as np
import pyspark
import os
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [3]:
spark = SparkSession.builder.appName("Movie Recommendation System").getOrCreate()

24/11/03 14:24:35 WARN Utils: Your hostname, sathvik-HP-EliteBook-x360-1030-G2 resolves to a loopback address: 127.0.1.1; using 192.168.0.108 instead (on interface wlp58s0)
24/11/03 14:24:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/03 14:24:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/03 14:24:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
df = spark.read.csv("../Data/ratings_small.csv", header=True, inferSchema=True)
df.show(5)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|     31|   2.5|1260759144|
|     1|   1029|   3.0|1260759179|
|     1|   1061|   3.0|1260759182|
|     1|   1129|   2.0|1260759185|
|     1|   1172|   4.0|1260759205|
+------+-------+------+----------+
only showing top 5 rows



In [7]:
df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [8]:
df.describe().show()

24/11/03 14:26:01 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+------------------+------------------+--------------------+
|summary|            userId|           movieId|            rating|           timestamp|
+-------+------------------+------------------+------------------+--------------------+
|  count|            100004|            100004|            100004|              100004|
|   mean| 347.0113095476181|12548.664363425463| 3.543608255669773|1.1296390869392424E9|
| stddev|195.16383797819535|26369.198968815268|1.0580641091070326|1.9168582602710962E8|
|    min|                 1|                 1|               0.5|           789652009|
|    max|               671|            163949|               5.0|          1476640644|
+-------+------------------+------------------+------------------+--------------------+



In [12]:
df = df.drop("timestamp")
df.show(3)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|     31|   2.5|
|     1|   1029|   3.0|
|     1|   1061|   3.0|
+------+-------+------+
only showing top 3 rows



In [13]:
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

In [14]:
als = ALS(
  maxIter=10,
  regParam=0.1,
  userCol="userId",
  itemCol="movieId",
  ratingCol="rating",
  coldStartStrategy="drop"
)

In [15]:
model = als.fit(train_data)

In [16]:
predictions = model.transform(test_data)
predictions.show(5)

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   148|    185|   3.0|  3.120598|
|   148|    364|   4.0| 4.0532117|
|   148|    596|   4.5| 3.8914142|
|   148|   1028|   5.0|   3.82561|
|   148|   1136|   4.5|  4.492236|
+------+-------+------+----------+
only showing top 5 rows



In [18]:
metrics = ["rmse", "mse", "mae"]

for metric in metrics:
    evaluator = RegressionEvaluator(
        metricName=metric,
        labelCol="rating",
        predictionCol="prediction"
	)
    result = evaluator.evaluate(predictions)
    print(f"{metric}: {result}")
    

rmse: 0.9140473737488305


mse: 0.8354826014571343


mae: 0.708307642911486
